# Sheet

In [64]:
pip install kneed

Note: you may need to restart the kernel to use updated packages.


In [65]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [66]:
import torch

In [67]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
import random
import pandas as pd
import warnings

import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as sm
import itertools
from tqdm import tqdm
import seaborn as sns
from kneed import KneeLocator
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean

## Part 1: Data Preparation 
### Adjust the bond prices to mitigate the impact of the benchmark bond roll.

In [68]:
tr_dm = pd.read_excel('GENERIC BOND PRICE.xlsx', sheet_name='DM_PRICE')
tr_dm.Dates = pd.to_datetime(tr_dm.Dates)
tr_dm = tr_dm.set_index('Dates')

In [69]:
def calculate_dirty_price(clean_price, coupon, days_since_last_coupon):
    daily_coupon = coupon / 365  
    accrued_interest = daily_coupon * days_since_last_coupon
    return clean_price + accrued_interest

def calculate_days_since_last_coupon(date, last_coupon_date):
    return (date - last_coupon_date).days

def find_first_coupon_date(coupon_series):
    coupon_changes = coupon_series.diff().dropna()
    if len(coupon_changes) > 0:
        first_change_date = coupon_changes.index[0]
        return first_change_date - pd.Timedelta(days=180)  # All coupon paid 6 months before the change
    else:
        return coupon_series.index[0] 
    

In [70]:
dirty_price_series = {}
for column in tr_dm.columns:
    if column.endswith('Govt'):
        clean_price_series = tr_dm[column]
        coupon_series = tr_dm[f"{column} CPN"]

        first_coupon_date = find_first_coupon_date(coupon_series)
        last_coupon_date = first_coupon_date
        current_coupon = coupon_series.iloc[0]
        
        dirty_prices = []
        
        for date, clean_price in clean_price_series.items():
            if coupon_series[date] != current_coupon:
                last_coupon_date = date - pd.Timedelta(days=180)
                current_coupon = coupon_series[date]
            
            days_since_last_coupon = calculate_days_since_last_coupon(date, last_coupon_date)
            
            dirty_price = calculate_dirty_price(clean_price, current_coupon, days_since_last_coupon)
            dirty_prices.append(dirty_price)

            if days_since_last_coupon >= 180:
                last_coupon_date = date

        tr_dm[f"{column}_Dirty"] = pd.Series(dirty_prices, index=clean_price_series.index)

In [71]:

for bond_col in tr_dm.columns:
    if bond_col.endswith('Dirty'): 
        coupon_col = bond_col.replace('Govt_Dirty', 'Govt CPN')
        bond_col = bond_col.replace('_Dirty','')
        
        tr_dm[f"{bond_col} Adjusted"] = tr_dm[bond_col]
        
        coupon_changes = tr_dm[coupon_col].diff().fillna(0) != 0

        for change_date in tr_dm.index[coupon_changes]:

            price_on_change = tr_dm.loc[change_date, bond_col]
            
            previous_price = tr_dm.loc[tr_dm.index[tr_dm.index.get_loc(change_date) - 1], bond_col]
            
            price_diff = price_on_change - previous_price
            
            tr_dm.loc[change_date:, f"{bond_col} Adjusted"] -= price_diff

        tr_dm[f"{bond_col} Adjusted Return"] = tr_dm[f"{bond_col} Adjusted"].pct_change()


# col_adj = [i for i in tr_dm.columns if (i.endswith('Adjusted') or i.endswith('Adjusted Return'))]

col_adj_price = [i for i in tr_dm.columns if i.endswith('Adjusted')]

col_adj_return = [i for i in tr_dm.columns if i.endswith('Adjusted Return')]

tr_dm_net = tr_dm[col_adj_return].fillna(0)

tr_dm_net_price = tr_dm[col_adj_price].fillna(0)

## Part 3: Identify Correlated Assets (Developed Market)
### Step1 : PCA (cluster by the first principal component)

In [72]:
from statsmodels.tsa.stattools import coint

def test_cointegration_in_clusters(data, cluster_dict):

    cointegrated_pairs = []
    for cluster_num in cluster_dict:
        asset_names = cluster_dict[cluster_num]
        asset_names = [i.replace(' Return', '') for i in asset_names]
        
        # Loop through each pair of assets in the cluster
        for i in range(len(asset_names)):
            for j in range(i + 1, len(asset_names)):
                asset1 = asset_names[i]
                asset2 = asset_names[j]
             
                series1 = data[asset1]
                series2 = data[asset2]
                
                # Perform the Engle-Granger cointegration test
                coint_t, p_value, _ = coint(series1, series2)
                
                # set a higher significant level (0.2) to avoid missing potential relationship
                if p_value < 0.2:  
                    cointegrated_pairs.append([asset1,asset2])
                else:
                    pass
                    # print(f"  {asset1} and {asset2} are NOT cointegrated (p-value: {p_value:.4f})")

    return cointegrated_pairs


In [73]:

def noncoherent_pair_cluster(data, data_price): # input: return data, price data
    # Step 1: Cluster by Principal component 1
    scaler = StandardScaler()
    asset_returns = pd.DataFrame(scaler.fit_transform(data), columns= data.columns)


    # Calculate the loadings of bond returns on PCs
    K = 1
    pca = PCA(n_components=K)
    pca.fit(asset_returns)
    loadings = pca.components_.T

    wcss = []
    for k in range(1, 10):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(loadings)
        # Inertia: Sum of squared distances to closest cluster center
        wcss.append(kmeans.inertia_)  
        
    # Use the KneeLocator to detect the elbow point
    kneedle = KneeLocator(range(1, 10), wcss, S=1.0, curve='convex', direction='decreasing')

    # Get the optimal number of clusters
    optimal_clusters = kneedle.elbow


    # Clustering in the principal component space and using K-means to cluster different govt bonds
    kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
    clusters = kmeans.fit_predict(loadings)
    asset_names = asset_returns.columns

    cluster_dic = {}
    for cluster in range(optimal_clusters):
        cluster_assets = asset_names[clusters == cluster]
        cluster_dic[cluster + 1] =cluster_assets


    # Step 2: Find cointegration pairs
    cointegrated_pairs = test_cointegration_in_clusters(data_price, cluster_dic)

    # Step 3: Exclude pairs have similar PC2 and PC3
    K = 3
    pca = PCA(n_components=K)
    pca.fit(asset_returns)
    loadings = pca.components_.T
    explained_var = pca.explained_variance_ratio_

    loadings_df = pd.DataFrame(loadings, columns = ['PC1','PC2','PC3'], index = data.columns)

    pc2_diff_list = [abs(loadings_df.loc[f"{i} Return", 'PC2'] - loadings_df.loc[f"{j} Return", 'PC2']) for (i,j) in cointegrated_pairs]

    threshold_pc2 = np.quantile(pc2_diff_list, 0.5)

    coherent_pair = []
    noncoherent_pair = []

    for i in range(len(cointegrated_pairs)):
        asset1 = cointegrated_pairs[i][0]
        asset2 = cointegrated_pairs[i][1]

        pc1_diff = abs(loadings_df.loc[f"{asset1} Return", 'PC1']  - loadings_df.loc[f"{asset2} Return", 'PC1'])
        pc2_diff = abs(loadings_df.loc[f"{asset1} Return", 'PC2']  - loadings_df.loc[f"{asset2} Return", 'PC2'])

        if pc2_diff < threshold_pc2:
            coherent_pair.append((f'{asset1}', f'{asset2}'))
            # print(asset1, asset2, pc1_diff, 'coherent')
        else:
            noncoherent_pair.append((f'{asset1}', f'{asset2}'))
            # print(asset1, asset2, pc1_diff, 'noncoherent')
            

    return coherent_pair, noncoherent_pair


## Part 4: Naiive Rule-based Trading Algorithm

The code below implements a rule-based pair trading strategy that is based on the 2nd principal component loadings of the asset pair to construct a market-neutral spread.

Key Steps:
1. Rolling PCA Calculation:

Option 1: PCA is applied on the returns of only two selected assets. In this case, the first principal component (PC1) represents the common trend between the two selected assets.

Option 2: PCA can be applied to all 11 bonds, which would make the PC1 represent the broader market trend for all treasury bonds. PC2 represents the specific factors that diverge from the broader market, allowing for more targeted trading opportunities.

2. Rolling Loadings:

The PCA loadings are calculated on a rolling basis to avoid look-ahead bias over time. The strategy focuses on the loadings for the second principal component (PC2), which reflects the bond-specific factors that are relatively insensitive (orthogonal) to broader market movements captured by PC1.
The code extracts the rolling loadings of the selected assets on PC2 and uses these loadings to construct the spread.

3. Spread Construction:

The spread is calculated as a linear combination of the two assets' returns weighted by their PC2 loadings.
Then the weights are normalized based on the total absolute weight of the two assets to ensure that the portfolio remains balanced.

$ \text{Spread}  = w1 * \text{Asset 1 Return} - w2 * \text{Asset 2 Return}$

$ w1 = 1 $


$ w2 = w1 * \frac{\sigma 1}{\sigma 2} * \frac{PC loading Asset 1}{PC Loading Asset 2} $



1. Trading Signals:

Buy Signal: Generated when the z-score of the spread (number of standard deviations the spread deviates from its rolling mean) falls below a specified lower threshold. This implies the spread has diverged significantly, and the strategy goes long on the spread, expecting a reversal.
Sell Signal: Generated when the z-score rises above the upper threshold, indicating the spread has widened significantly. The strategy goes short on the spread, expecting a convergence.
Exit Signal: The strategy closes the positions when the spread reverts to a level closer to the mean, as defined by a close threshold.

5. Z-Score and Thresholds:

The z-score is calculated based on the rolling mean and rolling standard deviation of the spread. This z-score is used to quantify the divergence of the spread from its historical average.
The thresholds (upper, lower, and close) dictate when the strategy enters and exits positions.

6. Performance Evaluation:

The code tracks the cumulative returns of the pair trading strategy over time. It shifts the positions by one day to avoid look-ahead bias when calculating the returns for the next day.
The final cumulative returns are plotted to visualize the strategy's performance for the selected asset pair.

In [74]:
def rolling_pca_loadings(data, window_size, num_components=2):
    rolling_loadings = []
    
    for i in range(window_size, len(data) + 1):
        window_data = data[i - window_size:i]
        pca = PCA(n_components=num_components)
        pca.fit(window_data)
        loadings = pca.components_.T  
        rolling_loadings.append(loadings)
        
    return np.array(rolling_loadings)

def diff_sign_pc_loadings(data, window_size):
    loadings = rolling_pca_loadings(data, window_size, num_components=2)
    result = []
    for matrix in loadings:
        first_col = matrix[:, 0]
        second_col = matrix[:, 1]
        
        if np.sign(first_col[0]) == np.sign(first_col[1]):
            result.append(first_col)
        else:
            result.append(second_col)
    
    result_array = np.array(result)
    
    return result_array

In [75]:
# Non-coherent pair
def rule_based_strategy_info(data, window_size, upper_threshold,close_threshold, asset1,asset2):

    # 1. Rolling PCA Calculation:
    # option1: apply pca on two assets: pc1 represents the common trend between asset1 and asset 2 only
    asset1 = f'{asset1} Return'
    asset2 = f'{asset2} Return'
    selected_asset_returns = data[[f'{asset1}', f'{asset2}']]
    
    # 2. Rolling Loadings:
    rolling_loadings = diff_sign_pc_loadings(selected_asset_returns, window_size)

    pc_loading_asset1 = rolling_loadings[:, 0] 
    pc_loading_asset2 = rolling_loadings[:, 1] 

    # 3. Spread Construction:
    
    sigma_asset1 = data[f'{asset1}'].rolling(window=window_size).std().dropna()
    sigma_asset2 = data[f'{asset2}'].rolling(window=window_size).std().dropna()
    pc_loading_asset1 = pd.Series(pc_loading_asset1, index = sigma_asset1.index)
    pc_loading_asset2 = pd.Series(pc_loading_asset2, index = sigma_asset2.index)

    sigma_ratio = sigma_asset1 / sigma_asset2
    
    w1 = 1
    w2 = (w1 * sigma_ratio * pc_loading_asset1)/pc_loading_asset2
    
    spread = w1 * data[f'{asset1}'] - w2 * data[f'{asset2}']

    # 4. Trading Signals:
    rolling_mean = spread.rolling(window=window_size).mean()
    rolling_std = spread.rolling(window=window_size).std()

    # 5. Z-Score and Thresholds:
    z_score = (spread - rolling_mean) / rolling_std
    
    lower_threshold = - upper_threshold
    
    positions = pd.DataFrame(index=data.index, columns=['Position','Holdings_w1','Holdings_w2'])
    
    # Enter signal (spread deviates from the mean)
    positions['Position'] = np.where(z_score > upper_threshold, -1, 0)
    positions['Position'] = np.where(z_score < lower_threshold, 1, positions['Position'])
    
    # Exit signal (spread reverts to the mean)
    positions['Position'] = np.where((z_score > - close_threshold) & (positions['Position'] == 1), 0, positions['Position'])
    positions['Position'] = np.where((z_score < close_threshold) & (positions['Position'] == -1), 0, positions['Position'])
    
    positions['Holdings_w1'] = positions['Position'] * w1
    positions['Holdings_w2'] = positions['Position'] * w2


    # 6. Performance Evaluation:
    positions_shifted = positions.shift(1)  
    returns = positions_shifted['Holdings_w1'] * data[f'{asset1}'] + positions_shifted['Holdings_w2'] * data[f'{asset2}']
    initial_investment = 1
    cumulative_returns = (1+ returns).cumprod() -1

    return z_score, spread, returns, cumulative_returns, positions

In [76]:
# Non-coherent pair
def rule_based_strategy(data, window_size, upper_threshold,close_threshold, asset1,asset2):

    # 1. Rolling PCA Calculation:
    # option1: apply pca on two assets: pc1 represents the common trend between asset1 and asset 2 only
    asset1 = f'{asset1} Return'
    asset2 = f'{asset2} Return'
    selected_asset_returns = data[[f'{asset1}', f'{asset2}']]
    
    # 2. Rolling Loadings:
    rolling_loadings = diff_sign_pc_loadings(selected_asset_returns, window_size)

    pc_loading_asset1 = rolling_loadings[:, 0] 
    pc_loading_asset2 = rolling_loadings[:, 1] 

    # 3. Spread Construction:
    
    sigma_asset1 = data[f'{asset1}'].rolling(window=window_size).std().dropna()
    sigma_asset2 = data[f'{asset2}'].rolling(window=window_size).std().dropna()
    pc_loading_asset1 = pd.Series(pc_loading_asset1, index = sigma_asset1.index)
    pc_loading_asset2 = pd.Series(pc_loading_asset2, index = sigma_asset2.index)

    sigma_ratio = sigma_asset1 / sigma_asset2
    
    w1 = 1
    w2 = (w1 * sigma_ratio * pc_loading_asset1)/pc_loading_asset2
    
    spread = w1 * data[f'{asset1}'] - w2 * data[f'{asset2}']

    # 4. Trading Signals:
    rolling_mean = spread.rolling(window=window_size).mean()
    rolling_std = spread.rolling(window=window_size).std()

    # 5. Z-Score and Thresholds:
    z_score = (spread - rolling_mean) / rolling_std
    
    lower_threshold = - upper_threshold
    
    positions = pd.DataFrame(index=data.index, columns=['Position','Holdings_w1','Holdings_w2'])
    
    # Enter signal (spread deviates from the mean)
    positions['Position'] = np.where(z_score > upper_threshold, -1, 0)
    positions['Position'] = np.where(z_score < lower_threshold, 1, positions['Position'])
    
    # Exit signal (spread reverts to the mean)
    positions['Position'] = np.where((z_score > - close_threshold) & (positions['Position'] == 1), 0, positions['Position'])
    positions['Position'] = np.where((z_score < close_threshold) & (positions['Position'] == -1), 0, positions['Position'])
    
    positions['Holdings_w1'] = positions['Position'] * w1
    positions['Holdings_w2'] = positions['Position'] * w2


    # 6. Performance Evaluation:
    positions_shifted = positions.shift(1)  
    returns = positions_shifted['Holdings_w1'] * data[f'{asset1}'] + positions_shifted['Holdings_w2'] * data[f'{asset2}']
    initial_investment = 1
    cumulative_returns = (1+ returns).cumprod() -1

    return cumulative_returns[-1]

### Grid search for the optimal parameters

In [77]:
def optimize_parameters(data, noncoherent_pair, window_size_list, upper_threshold_list, close_threshold_list):

    param_results = {}
    pair_results = {}
    
    # Try each parameter combination
    for window_size, upper_threshold, close_threshold in itertools.product(
            window_size_list, upper_threshold_list, close_threshold_list):
        
        if close_threshold >= upper_threshold:
            continue
            
        returns = []
        pair_details = []
        
        for asset_pair in noncoherent_pair:
            asset1 = asset_pair[0]#.replace(' Adjusted Return', '')
            asset2 = asset_pair[1]#.replace(' Adjusted Return', '')
            
            # Get return for this pair with current parameters
            result = rule_based_strategy(data, window_size, upper_threshold, close_threshold, asset1, asset2)
            returns.append(result)
            
            pair_details.append({'asset1': asset1, 'asset2': asset2, 'return': result})
        
        # Store average return for this parameter combination
        param_key = (window_size, upper_threshold, close_threshold)
        param_results[param_key] = {'avg_return': np.mean(returns),'pair_details': pair_details}

    
    # Find best parameter combination
    best_params = max(param_results.items(), key=lambda x: x[1]['avg_return'])
    window_size, upper_threshold, close_threshold = best_params[0]
    
    
    return upper_threshold, close_threshold, window_size


## Part 5: Reinforcement Learning Based Strategy

To enhance the existing rule-based strategy for pair trading using a Deep Q-Network (DQN), we can replace the rule-based signals with a reinforcement learning (RL) approach. The purpose for RL step is to find 'when' to trade and 'how much' to trade by interacting with observation space, action space and reward space.

Key Points:

Teachnique: DQN Strategy

Dynamic Pair Selection & Training Process
1. Formation Period (Year T): Select correlated pairs using PCA
2. Training Period (Year T+1): Train RL agent on selected pairs
3. Testing Period (Year T+2): Test agent on newly generated pairs
4. Repeat process by rolling forward one year for robust validation

![image.png](attachment:./attachment:853841c4-393a-4b7c-870b-290257aa81f2.png)

Model Architecture
1. Deep Q-Network (DQN) for automated trading decisions 
2. State space includes normalized spread, z-score, position metrics
3. Action space: Long (1), Neutral (0), Short (-1) positions
4. Reward = Return - beta * (RL action - rule-based strategy action)

Training Process:
1. the agent learns optimal entry and exit points through extensive episodes with different pairs and market conditions
2. Set the network with highest reward as target network, agent continuously learns to match or exceed this taret network performance
3. the agent develop resilience to both regime changes and switching between different asset pairs, making it truly adaptive to market dynamics.

In [105]:
# defines a new class called DQN that inherits from PyTorch's nn.Module. 

import torch
import torch.nn as nn
import torch.nn.functional as F

# todo: to be fine-tuned
class DQN(nn.Module):
    def __init__(self, state_size, action_size, hidden_sizes,activation = 'relu', dropout_rate=0.2):
        super(DQN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if activation == 'relu':
            act_fn = nn.ReLU()
        elif activation == 'leaky_relu':
            act_fn = nn.LeakyReLU(0.01)
        elif activation == 'elu':
            act_fn = nn.ELU()
        elif activation == 'selu':
            act_fn = nn.SELU()
        elif activation == 'tanh':
            act_fn = nn.Tanh()

    
        # Input layer
        # Input layer with proper initialization
        input_layer = nn.Linear(state_size, hidden_sizes[0])
        nn.init.xavier_uniform_(input_layer.weight)
        nn.init.zeros_(input_layer.bias)
        self.layers.append(input_layer)
        self.layers.append(act_fn)
        
        # Hidden layers with proper initialization
        for i in range(1, len(hidden_sizes)):
            hidden_layer = nn.Linear(hidden_sizes[i-1], hidden_sizes[i])
            nn.init.xavier_uniform_(hidden_layer.weight)
            nn.init.zeros_(hidden_layer.bias)
            self.layers.append(hidden_layer)
            self.layers.append(act_fn)
            
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(dropout_rate))
        
        # Output layer with proper initialization
        output_layer = nn.Linear(hidden_sizes[-1], action_size)
        nn.init.xavier_uniform_(output_layer.weight)
        nn.init.zeros_(output_layer.bias)
        self.layers.append(output_layer)

    def forward(self, state):
        x = state
        for i, layer in enumerate(self.layers):
            x = layer(x)
        return x
    
# creates a named tuple to store experience replays. a convenient way to group related data.
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'done'))

In [79]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)
    
    def push(self, *args):
        self.memory.append(Transition(*args))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [106]:
class DQNAgent:
    def __init__(self, state_size, action_size,hidden_size, learning_rate=1e-5):
        self.hidden_size= hidden_size
        self.state_size = state_size
        self.action_size = action_size
        self.memory = ReplayBuffer(10000) # a replay buffer with a capacity of 10,000 experiences
        self.gamma = 0.95    # discount factor
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.995
        self.learning_rate = learning_rate
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.policy_net = DQN(state_size, action_size,hidden_size).to(self.device)
        self.target_net = DQN(state_size, action_size,hidden_size).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        self.batch_size = 32

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate) #update for policy network's weight
        self.training_rmse_history = []  # Track training RMSE
        self.q_value_history = []  # Store Q-value predictions
        self.actual_returns_history = []  # Store actual returns
        self.next_states = []

        
    def act(self, state, store_next_state = None):
        if random.random() <= self.epsilon: # exploration (random act)
            self.q_value_history.append(0)
            return random.randrange(self.action_size)
        
        with torch.no_grad(): 

            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            action_values = self.policy_net(state)
            action = action_values.max(1)[1].item()

            q_value = action_values[0][action].item()

            if not np.isnan(q_value):
                self.q_value_history.append(q_value)
            else:
                self.q_value_history.append(0)

            return action


    def step(self, state, action, reward, next_state, done):
        self.memory.push(state, action, reward, next_state, done) # add new experience to replay buffer

    def learn(self):

        if len(self.memory) < self.batch_size: # check if enough space in buffer
            return
        
        transitions = self.memory.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.FloatTensor(batch.state).to(self.device)
        action_batch = torch.LongTensor(batch.action).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch.reward).to(self.device)
        next_state_batch = torch.FloatTensor(np.array(batch.next_state)).to(self.device)


        # pass our batch of states through our policy network to get Q-values for all actions. 
        # use gather to select only the Q-values for the actions that were actually taken. This gives us our current estimate of the Q-values for our sampled state-action pairs
        current_q_values = self.policy_net(state_batch).gather(1, action_batch)

        # For each non-terminal next state, we compute the maximum Q-value using our target network. 
        # We use the target network (which is updated less frequently) to provide a more stable target for learning.
        next_states = torch.FloatTensor([s for s in batch.next_state if s is not None]).to(self.device)

        with torch.no_grad():
            if next_states.size(0) > 0:
                next_actions = self.policy_net(next_states).max(1)[1]
                next_state_values = self.target_net(next_states).gather(1, next_actions.unsqueeze(1))
            else:
                next_state_values = torch.zeros_like(current_q_values)

        expected_q_values = (next_state_values * self.gamma) + reward_batch.unsqueeze(1)


        # Weighted loss
        mse_loss = F.mse_loss(current_q_values, expected_q_values)
        # loss =  F.smooth_l1_loss(current_q_values, expected_q_values)
        training_rmse  = torch.sqrt(mse_loss)

        self.optimizer.zero_grad()
        mse_loss.backward()
        self.optimizer.step()

        self.training_rmse_history.append(training_rmse.item()) 

        return training_rmse.item()



    # as we have more training samples, we don't need to explore too often
    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def calculate_testing_rmse(self):
        if not self.next_states:
            return None
            
        with torch.no_grad():
 
            next_state = torch.FloatTensor(self.next_states).to(self.device)
            next_actions = self.policy_net(next_state).max(1)[1]
            next_state_values = self.target_net(next_state).gather(1, next_actions.unsqueeze(1))

            rewards = torch.FloatTensor(self.actual_returns_history).to(self.device)

            target_q_values = rewards + (self.gamma * next_state_values.squeeze())
            
            predicted_q_values = torch.FloatTensor(self.q_value_history).to(self.device)
        
            bellman_error = torch.sqrt(F.mse_loss(predicted_q_values, target_q_values))
            
            return bellman_error.item()

    
    def reset_test_history(self):

        self.q_value_history = []
        self.actual_returns_history = []
        self.next_states = []

In [119]:
class MultiPairTradingEnv:
    def __init__(self, asset_returns_raw, pair_list, upper_threshold, close_threshold, window_size, beta):
        self.asset_returns_raw = asset_returns_raw
        self.pair_list = pair_list  
        self.window_size = window_size
        self.upper_threshold = upper_threshold
        self.close_threshold = close_threshold
        self.beta = beta
        self.action_space = [-1, 0, 1]
        
        # Initialize spreads for all pairs
        self.spreads = {}
        self.w2s = {}
        for asset1, asset2 in pair_list:
            spread, w2 = self._calculate_spread(asset1, asset2)
            self.spreads[(asset1, asset2)] = spread
            self.w2s[(asset1, asset2)] = w2
        
        # Initialize training variables
        self.reset()

    def _calculate_spread(self, asset1, asset2):
        """Calculate spread for a given pair"""
        selected_asset_returns = self.asset_returns_raw[[f'{asset1} Return', f'{asset2} Return']]
        
        rolling_loadings = diff_sign_pc_loadings(selected_asset_returns, self.window_size)
        pc_loading_asset1 = rolling_loadings[:, 0]
        pc_loading_asset2 = rolling_loadings[:, 1]
        
        sigma_asset1 = self.asset_returns_raw[f'{asset1} Return'].rolling(window=self.window_size).std().dropna()
        sigma_asset2 = self.asset_returns_raw[f'{asset2} Return'].rolling(window=self.window_size).std().dropna()
        
        pc_loading_asset1 = pd.Series(pc_loading_asset1, index=sigma_asset1.index)
        pc_loading_asset2 = pd.Series(pc_loading_asset2, index=sigma_asset2.index)
        sigma_ratio = sigma_asset1 / sigma_asset2
        
        w1 = 1
        w2 = (w1 * sigma_ratio * pc_loading_asset2) / pc_loading_asset1
        total_weight = abs(w1) + abs(w2)
        w1 = w1 / total_weight
        w2 = w2 / total_weight

        spread = w1 * self.asset_returns_raw[f'{asset1} Return'] - w2 * self.asset_returns_raw[f'{asset2} Return']
        
        return spread, w2

    def reset(self,ifbacktest = False, pair = []):
        # Randomly select a pair
        if ifbacktest:
            self.current_pair = pair
        else:
            self.current_pair = random.choice(self.pair_list)
        self.asset1, self.asset2 = self.current_pair
        # Get spread and w2 for selected pair
        self.spread = self.spreads[self.current_pair]
        self.w2 = self.w2s[self.current_pair]
        
        # Reset position and portfolio
        self.position = 0
        self.previous_position = 0
        self.portfolio_value = 1
        self.steps_taken = 0
        self.cumulative_return = [self.portfolio_value]
        # Reset tracking variables
        self.portfolio_start_value = self.portfolio_value
        self.last_position_change = None
        self.position_start_value = None
        self.return_list = []
        self.action_history = []
        self.reward = 0
        
        self.current_step = self.window_size
        
        return self._get_normalized_state()

    def step(self, action):
        self.steps_taken += 1
        self.previous_position = self.position
        new_position = self.action_space[action]
        
        # Handle position changes
        portfolio_reward = 0
        if new_position != 0 and self.previous_position == 0:
            self.last_position_change = self.current_step
            self.position_start_value = self.portfolio_value
        elif new_position == 0 and self.previous_position != 0:
            self.last_position_change = None
            self.position_start_value = None
            portfolio_reward = self._calculate_portfolio_reward()

        self.position = new_position
        
        # Calculate rewards
        spread_return = self._calculate_immediate_reward()
        baseline_action = self._get_baseline_action()
        deviation_penalty = self.beta * abs(new_position - baseline_action)
        self.reward = portfolio_reward - deviation_penalty

        # Update portfolio value
        self.return_list.append(1 + spread_return)
        self.cumulative_return = np.cumprod(self.return_list) -1
        self.portfolio_value = self.cumulative_return[-1]
        
        # Store action
        self.action_history.append({
            'step': self.current_step,
            'pair': self.current_pair,
            'position': self.position,
            'reward': self.reward,
            'spread_return': spread_return,
            'portfolio_value': self.portfolio_value,
            'cumulative_return': self.cumulative_return
        })
        
        # Advance step
        self.current_step += 1
        
        # Episode ends after 1000 steps or at end of data
        done = self.current_step >= len(self.spread) - 1
        
        # Close position at end of episode
        if done and self.position != 0:
            self.position = 0
            portfolio_reward = self._calculate_portfolio_reward()
            self.reward += portfolio_reward
        
        return self._get_normalized_state(), self.reward, done, {
            'pair': self.current_pair,
            'portfolio_value': self.portfolio_value,
            'spread_return': spread_return,
            'deviation_penalty': deviation_penalty
        }

    def _get_normalized_state(self):
        """Get normalized state representation"""
        spread = self.spread.iloc[self.current_step]
        z_score = self._calculate_z_score(spread)
        
        return np.array([
            self.position, 
            spread,
            z_score
        ])


    def _calculate_z_score(self, spread):
        if self.current_step < self.window_size:
            return 0  
        window = self.spread.iloc[self.current_step - self.window_size:self.current_step]
        std_dev = window.std()
        if std_dev == 0 or np.isnan(std_dev):
            return 0  
        return (spread - window.mean()) / std_dev

    def _calculate_portfolio_reward(self):
        if self.position_start_value is None:
            return 0
        return (self.portfolio_value - self.position_start_value) / self.position_start_value

    def _calculate_immediate_reward(self):
        asset1_return = self.asset_returns_raw[f'{self.asset1} Return'].iloc[self.current_step]
        asset2_return = self.asset_returns_raw[f'{self.asset2} Return'].iloc[self.current_step]
        prev_index = self.asset_returns_raw.index[self.current_step-1]
        w2_prev = self.w2.loc[prev_index]
        
        spread_return = asset1_return - w2_prev * asset2_return

        
        return self.previous_position * spread_return
    
    def _get_baseline_action(self):
        z_score = self._calculate_z_score(self.spread.iloc[self.current_step])
        lower_threshold = -1 * self.upper_threshold
        
        if z_score > self.upper_threshold and self.position < 0:  # short signal
            return -1
        elif z_score < lower_threshold and self.position > 0:  # long signal
            return 1
        elif abs(z_score) < self.close_threshold and self.position == 0:  # neutral
            return 0
        
        return self.position  
    

In [107]:
def train_multi_pair_dqn(env, episodes, hidden_size, batch_size=32):
    state_size = 3  # position, spread, z-score
    action_size = len(env.action_space)
    
    agent = DQNAgent(state_size, action_size, hidden_size)
    
    # Initialize training metrics for each pair
    training_metrics = {

        'pair_performance': {pair: {
            'rewards': [],
            'portfolio_values': [],
            'best_portfolio_value': float('-inf'),
            'best_model_rmse': float('-inf'),
            'best_reward': float('-inf'),
            'best_cumulative_return':[],
            'best_state_dict': None
        } for pair in env.pair_list}
    }

    for episode in range(episodes):
        state = env.reset()
        current_pair = env.current_pair
        episode_reward = 0
        done = False
        episode_training_rmse = []
        agent.reset_test_history()
        while not done:
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            
            agent.memory.push(state, action, reward, next_state, done)
            if len(agent.memory) > batch_size:
                training_rmse = agent.learn()
                if training_rmse is not None:
                    episode_training_rmse.append(training_rmse)

            # Store Q-value prediction and actual return for testing RMSE
            agent.actual_returns_history.append(reward)
            
            episode_reward += reward
            state = next_state

        return_qvalue_rmse = agent.calculate_testing_rmse()
        
        # Store pair-specific metrics
        pair_metrics = training_metrics['pair_performance'][current_pair]
        pair_metrics['rewards'].append(episode_reward)
        pair_metrics['portfolio_values'].append(env.portfolio_value)
        
        # Update best model for current pair if performance improves
        if episode_reward > pair_metrics['best_reward']:
            pair_metrics['best_portfolio_value'] = env.portfolio_value
            pair_metrics['best_reward'] = episode_reward
            pair_metrics['best_state_dict'] = agent.policy_net.state_dict()
            pair_metrics['best_cumulative_return'] = env.cumulative_return
            current_rmse = np.mean(episode_training_rmse) if episode_training_rmse else float('inf')
            pair_metrics['best_model_rmse'] = current_rmse
            # pair_metrics['best_model_rmse'] = return_qvalue_rmse

            # Update target network with best model
            agent.target_net.load_state_dict(pair_metrics['best_state_dict'])

        agent.decay_epsilon()

    # Find the best performing model across all pairs
    best_pair = max(training_metrics['pair_performance'].items(),
                   key=lambda x: x[1]['best_reward'])[0]
    best_metrics = training_metrics['pair_performance'][best_pair]
    
    agent.target_net.load_state_dict(best_metrics['best_state_dict'])
    
    return agent, training_metrics

def backtest(env, agent, target_pair):
    state = env.reset(True, target_pair)
    done = False
    agent.reset_test_history()

    while not done:
        
        action = agent.act(state)

        next_state, reward, done, _ = env.step(action)

        agent.actual_returns_history.append(reward)
  
        if next_state is not None: # new added
            agent.next_states.append(next_state)
        else:
            agent.next_states.append(0)

        state = next_state

    # Calculate testing RMSE for the backtest
    testing_rmse = agent.calculate_testing_rmse()

    backtest_results = {
        'action_history': env.action_history,
        'testing_rmse': testing_rmse
    }

    return backtest_results

In [83]:

def plot_cumulative_returns(dates, training_returns, pair, type):

    plt.figure(figsize=(12, 6))
    plt.plot(dates, training_returns, label='Returns')
    plt.title(f'Cumulative Returns for {pair[0]} and {pair[1]} in {type} set')
    plt.xlabel('Dates')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def get_last_cumulative_return(dates, action_results, target_pair):
    action_history = action_results['action_history']

    pair_actions = [action for action in action_history if action['pair'] == target_pair]
    
    if pair_actions:
        last_action = pair_actions[-1]
        cumulative_return = last_action['cumulative_return']
        # plot_cumulative_returns(dates, cumulative_return, target_pair, 'Testing')
        rmse = action_results['testing_rmse']
        mean_return, sharpe = get_summary(cumulative_return)

        return {
        'Mean Return (%) Annual': "{:.4f}".format(mean_return),
        'Sharpe Ratio Annual': "{:.4f}".format(sharpe),
        'RMSE':"{:.4f}".format(rmse)
    }
    else:
        return None
    
def get_summary(cumulative_return):

    periodic_returns = (1 + cumulative_return[1:]) / (1+cumulative_return[:-1]) - 1
    periodic_returns = periodic_returns[~np.isinf(periodic_returns) & ~np.isneginf(periodic_returns)]
    periodic_returns = periodic_returns[~np.isnan(periodic_returns)]
    # Mean return
    mean_return = np.mean(periodic_returns) *252 * 100
    
    # Sharpe ratio (assuming risk-free rate = 0)
    sharpe = (np.mean(periodic_returns) / np.std(periodic_returns))* np.sqrt(252)
    
    # Maximum drawdown
    cumulative = np.array(cumulative_return)
    running_max = np.maximum.accumulate(cumulative)


    return mean_return, sharpe


In [116]:
# train model pair by pair

if __name__ == "__main__":
    
    start_year = 2014
    end_year = 2024
    summary_data = []
  
    training_testing_Split = '2018-12-31'
    train_period = tr_dm_net[tr_dm_net.index <= training_testing_Split]
    formation_period = train_period
    test_period = tr_dm_net[tr_dm_net.index > training_testing_Split]

    formation_period_price = tr_dm_net_price[tr_dm_net_price.index <= training_testing_Split]
    train_period_price = tr_dm_net_price[tr_dm_net_price.index > training_testing_Split]


    coherent_pairs_in_formation, noncoherent_pair = noncoherent_pair_cluster(formation_period, formation_period_price)

    window_size_list = [30, 60]
    upper_threshold_list = [1.0, 1.25, 1.5,1.75]
    close_threshold_list = [0.1 , 0.2]

    upper_threshold, close_threshold, window_size = optimize_parameters(train_period, coherent_pairs_in_formation, window_size_list, upper_threshold_list,close_threshold_list)
    # upper_threshold = 1.0
    # close_threshold = 0.1
    # window_size = 60
    # beta = 0.1


    hidden_size_list = [[128,128],[128,64],[2],[5,5],[5,2],[5],[2,2,2]]

    for pair in coherent_pairs_in_formation:
        env = MultiPairTradingEnv(train_period, [pair], upper_threshold, close_threshold, window_size, beta)

        for hidden_size in hidden_size_list:
            
            train_agent, train_env = train_multi_pair_dqn(env, episodes=100, hidden_size = hidden_size)

            training_dates = train_period.index[window_size+1:]

            best_cumulative_return = train_env['pair_performance'][pair]['best_cumulative_return']
            rmse = train_env['pair_performance'][pair]['best_model_rmse']
            # plot_cumulative_returns(training_dates, best_cumulative_return, pair, 'Training')
            mean_return, sharpe = get_summary(best_cumulative_return)

            summary_training = {'Mean Return (%) Annual': "{:.4f}".format(mean_return),
                                'Sharpe Ratio Annual': "{:.4f}".format(sharpe),'RMSE':"{:.4f}".format(rmse)}
            
            summary_training['Pair'] = f"{pair[0]}-{pair[1]}"
            summary_training['Data Type'] = 'Train Data'
            summary_training['Hidden Size'] = '-'.join(str(layer) for layer in hidden_size)

            summary_data.append(summary_training)

            # Backtest

            env_test = MultiPairTradingEnv(test_period, [pair], upper_threshold, close_threshold, window_size, beta)
            testing_dates = test_period.index[window_size+1:]

            backtest_results = backtest(env_test, train_agent, pair)
            summary_testing = get_last_cumulative_return(testing_dates, backtest_results, pair)
            summary_testing['Pair'] = f"{pair[0]}-{pair[1]}"
            summary_testing['Data Type'] = 'Test Data'
            summary_testing['Hidden Size'] = '-'.join(str(layer) for layer in hidden_size)

            summary_data.append(summary_testing)

    summary_df = pd.DataFrame(summary_data)

    columns = ['Data Type', 'Hidden Size', 'Pair', 'Mean Return (%) Annual', 'Sharpe Ratio Annual','RMSE']
    summary_df = summary_df[columns]
    


    


In [117]:
pivot_df = pd.pivot_table(
   summary_df,
   index=['Hidden Size', 'Pair'],
   columns='Data Type',
   values=['Mean Return (%) Annual', 'Sharpe Ratio Annual','RMSE'],
   aggfunc='first' 
).round(4)
pivot_df.head(60)

Mean Return (%) Annual 
 RMSE 
 Sharpe Ratio Annual 
 
 
 
 Data Type 
 Test Data 
 Train Data 
 Test Data 
 Train Data 
 Test Data 
 Train Data 
 
 
 Hidden Size 
 Pair 
 
 
 
 
 
 
 
 
 
 
 128-128 
 GTAUD10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 0.7632 
 -1.3551 
 0.0460 
 0.0454 
 0.1458 
 -0.3433 
 
 
 GTFRF10Y Govt Adjusted-GTSEK10Y Govt Adjusted 
 -3.1340 
 1.2429 
 0.0112 
 0.0172 
 -0.7640 
 0.4125 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 2.8136 
 -1.1739 
 0.1093 
 0.0810 
 0.7294 
 -0.3457 
 
 
 GTSEK10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 -1.0854 
 -0.0349 
 0.0079 
 0.0395 
 -0.2958 
 -0.0123 
 
 
 128-64 
 GTAUD10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 0.6224 
 -1.5462 
 0.0154 
 0.0376 
 0.1291 
 -0.3728 
 
 
 GTFRF10Y Govt Adjusted-GTSEK10Y Govt Adjusted 
 -0.7632 
 -1.1051 
 0.0885 
 0.0386 
 -0.1878 
 -0.3665 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 -3.7500 
 -2.1861 
 0.0156 
 0.1244 
 -0.9418 
 -0.6386 
 
 
 GTSEK10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 -1.0889 
 -0.3130 
 0.0044 
 0.0343 
 -0.2993 
 -0.1100 
 
 
 2 
 GTAUD10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 2.1849 
 -1.4602 
 0.1394 
 0.3331 
 0.4271 
 -0.3499 
 
 
 GTFRF10Y Govt Adjusted-GTSEK10Y Govt Adjusted 
 0.6049 
 -0.2010 
 0.7759 
 0.6066 
 0.1474 
 -0.0664 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 -0.8909 
 0.3154 
 0.1547 
 0.4272 
 -0.2399 
 0.0906 
 
 
 GTSEK10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 2.3931 
 0.0251 
 0.4754 
 0.7714 
 0.6473 
 0.0091 
 
 
 2-2-2 
 GTAUD10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 -2.7952 
 -0.5051 
 0.0020 
 0.0010 
 -0.5609 
 -0.1201 
 
 
 GTFRF10Y Govt Adjusted-GTSEK10Y Govt Adjusted 
 -2.7889 
 -0.0055 
 0.0009 
 0.0016 
 -0.6777 
 -0.0018 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 -1.3337 
 0.8738 
 0.1366 
 0.1220 
 -0.3453 
 0.2475 
 
 
 GTSEK10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 -0.2176 
 -0.3933 
 0.0012 
 0.0066 
 -0.0592 
 -0.1295 
 
 
 5 
 GTAUD10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 0.8838 
 -1.7047 
 0.6780 
 1.0870 
 0.1727 
 -0.4014 
 
 
 GTFRF10Y Govt Adjusted-GTSEK10Y Govt Adjusted 
 -3.4037 
 -0.9779 
 0.1323 
 0.2080 
 -0.8435 
 -0.3201 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 -1.2965 
 1.9973 
 0.4096 
 0.3865 
 -0.3381 
 0.6050 
 
 
 GTSEK10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 -0.6721 
 0.3069 
 0.4214 
 0.6709 
 -0.1744 
 0.1103 
 
 
 5-2 
 GTAUD10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 2.3909 
 -0.7401 
 0.1264 
 0.1840 
 0.4573 
 -0.1825 
 
 
 GTFRF10Y Govt Adjusted-GTSEK10Y Govt Adjusted 
 1.1490 
 0.6452 
 0.1401 
 0.1722 
 0.2719 
 0.2209 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 -2.0456 
 1.1334 
 0.0926 
 0.1795 
 -0.5139 
 0.3325 
 
 
 GTSEK10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 1.1222 
 -0.1872 
 0.1446 
 0.5330 
 0.2995 
 -0.0668 
 
 
 5-5 
 GTAUD10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 0.1714 
 2.4262 
 0.2839 
 0.5061 
 0.0336 
 0.5920 
 
 
 GTFRF10Y Govt Adjusted-GTSEK10Y Govt Adjusted 
 2.4985 
 -0.5422 
 0.1416 
 0.2306 
 0.6189 
 -0.1824 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 0.2843 
 0.6685 
 0.0611 
 0.0976 
 0.0732 
 0.2017 
 
 
 GTSEK10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 0.0720 
 1.2507 
 0.3231 
 0.4087 
 0.0192 
 0.4654

# Sheet 2

## Appendix: Coherent pair and noncoherent pair comparison

In [86]:
# Training/testing data split
training_testing_Split = '2018-12-31'
training_return = tr_dm_net[tr_dm_net.index <= training_testing_Split]
testing_return = tr_dm_net[tr_dm_net.index > training_testing_Split]

training_price = tr_dm_net_price[tr_dm_net_price.index <= training_testing_Split]
testing_price = tr_dm_net_price[tr_dm_net_price.index > training_testing_Split]


In [87]:

def evaluate_pair_performance(pairs_list, data, window_size, upper_threshold, close_threshold):
    pairs_metrics = {}
    
    for asset1, asset2 in pairs_list:

        z_score, spread, returns, cumulative_return, positions = rule_based_strategy_info(data, window_size, upper_threshold, close_threshold, asset1, asset2)

        metrics = {
            'Cumulative Return': cumulative_return[-1],
            # 'Max Drawdown': calculate_max_drawdown(returns),
            'Spread Volatility': spread.std(),
            'signal_count': len(z_score[abs(z_score) > upper_threshold]),
            'avg_deviation': abs(z_score).mean(),
            'max_deviation': abs(z_score).max(),
            'signal_frequency': len(z_score[abs(z_score) > upper_threshold]) / len(z_score),
            'Average Trade Return': np.mean(returns[returns != 0])
        }

        pairs_metrics[(asset1.replace("Adjusted Return",""), asset2.replace("Adjusted Return",""))] = metrics
            
    return pairs_metrics

def compare_coherent_noncoherent(coherent_pair, noncoherent_pair, data, 
                                 window_size_coherent, upper_threshold_coherent, close_threshold_coherent,
                                 window_size_noncoherent, upper_threshold_noncoherent, close_threshold_noncoherent):

    coherent_metrics = evaluate_pair_performance(coherent_pair, data, 
                                               window_size_coherent, upper_threshold_coherent, close_threshold_coherent)
    
    noncoherent_metrics = evaluate_pair_performance(noncoherent_pair, data, 
                                                window_size_noncoherent, upper_threshold_noncoherent, close_threshold_noncoherent)
    
    
    return coherent_metrics, noncoherent_metrics


In [110]:
window_size_list = [30, 60, 90]
upper_threshold_list = [1.0, 1.25, 1.5]
close_threshold_list = [0.1, 0.5, 1]

coherent_pair, noncoherent_pair = noncoherent_pair_cluster(training_return, training_price)
# upper_threshold_coherent=upper_threshold_noncoherent = 1.0
# close_threshold_coherent = close_threshold_noncoherent = 0.1
# window_size_coherent = window_size_noncoherent = 60


upper_threshold_coherent, close_threshold_coherent, window_size_coherent = optimize_parameters(training_return, coherent_pair, window_size_list, upper_threshold_list, close_threshold_list)
upper_threshold_noncoherent, close_threshold_noncoherent, window_size_noncoherent = optimize_parameters(training_return, noncoherent_pair, window_size_list, upper_threshold_list, close_threshold_list)

coherent_metrics, noncoherent_metrics= compare_coherent_noncoherent(coherent_pair, noncoherent_pair,training_return, 
                                window_size_coherent, upper_threshold_coherent, close_threshold_coherent,
                                window_size_noncoherent, upper_threshold_noncoherent, close_threshold_noncoherent)

test_coherent_metrics, test_noncoherent_metrics= compare_coherent_noncoherent(coherent_pair, noncoherent_pair,testing_return, 
                                window_size_coherent, upper_threshold_coherent, close_threshold_coherent,
                                window_size_noncoherent, upper_threshold_noncoherent, close_threshold_noncoherent)


coherent_df = pd.DataFrame(coherent_metrics).round(4).T
print("\n" + "="*80)
print("TRAIN COHERENT PAIRS PERFORMANCE".center(80))
print("="*80)
print(coherent_df.to_string())

noncoherent_df = pd.DataFrame(noncoherent_metrics).round(4).T
print("\n" + "="*80)
print("TRAIN NONCOHERENT PAIRS PERFORMANCE".center(80))
print("="*80)
print(noncoherent_df.to_string())


test_coherent_df = pd.DataFrame(test_coherent_metrics).round(4).T
print("\n" + "="*80)
print("TEST COHERENT PAIRS PERFORMANCE".center(80))
print("="*80)
print(test_coherent_df.to_string())

test_noncoherent_df = pd.DataFrame(test_noncoherent_metrics).round(4).T
print("\n" + "="*80)
print("TEST NONCOHERENT PAIRS PERFORMANCE".center(80))
print("="*80)
print(test_noncoherent_df.to_string())


                        TRAIN COHERENT PAIRS PERFORMANCE                        
                                               Cumulative Return  Spread Volatility  signal_count  avg_deviation  max_deviation  signal_frequency  Average Trade Return
GTFRF10Y Govt Adjusted GTSEK10Y Govt Adjusted             0.3214             0.0062         201.0         0.7407         5.2811            0.1701                0.0016
GTSEK10Y Govt Adjusted GTCHF10Y Govt Adjusted             5.2274             0.0105         219.0         0.7599         4.8665            0.1853                0.0178
                       GTDEM10Y Govt Adjusted             0.0361             0.0027         208.0         0.7611         3.9946            0.1760                0.0002
GTAUD10Y Govt Adjusted GTNZD10Y Govt Adjusted            -0.0891             0.0058         210.0         0.7613         3.6757            0.1777               -0.0003

                      TRAIN NONCOHERENT PAIRS PERFORMANCE                    